In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pylab as pl

import os
import sys

import numpy as np
from scipy import ndimage as ni
import maxflow

from tifffile import imread, imsave
import cPickle as pickle

from skimage.filters import gaussian_filter

from spimagine import volshow

%reload_ext autoreload
%autoreload 2

%matplotlib qt

# %matplotlib inline
# %pylab inline
# pylab.rcParams['figure.figsize'] = (5, 5)

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


### Loading image to be segmented

In [6]:
filename = '/Volumes/FastData/ProjectGoldstein/Maddox_LITE/mneongreen_membrane_488_90percentpower_nobinning_60xWI_5msexposure_072916_single_timepoint_crop.tif'

sigma = [1,3,3]
image = imread(filename)
image_smooth = gaussian_filter(image,sigma)
print 'Image dimensions: ', image.shape
print 'Image min/max:    ', np.min(image), np.max(image)

Image dimensions:  (41, 457, 596)
Image min/max:     185 262


### Load sampled unit sphere and set parameters

In [90]:
plt.rcParams['figure.figsize'] = (5, 5)

# load pickeled unit sphere sampling
with open('sphere_sampling.pkl','r') as f:
    dictSphereData = pickle.load(f)
    
# sampling parameters
num_columns = dictSphereData['n']
col_vectors = dictSphereData['points']
neighbors = dictSphereData['neighbors']
neighbors_of = dictSphereData['neighbors_of']
num_neighbors = len(neighbors)/num_columns
K = 30       # as in Wu & Chen 2002

# net surface related parameters
max_delta_k = 4

# positioning of net in image

# A SWEET SET OF PARAMS
# center_x = 160
# center_y = 190
# center_z = 7
# max_r_x = 40
# max_r_y = 40
# max_r_z = 12

center_x = 210
center_y = 300
center_z = 26
max_r_x = image.shape[2] / 4
max_r_y = image.shape[1] / 4
max_r_z = image.shape[0] / 2
min_r_x = 20
min_r_y = 20
min_r_z = 3

In [91]:
volwin = volshow(image, stackUnits = [1.,1.,4.], raise_window=False)
# volwin = volshow(image_smooth, stackUnits = [1.,1.,4.], raise_window=False)

# Building Graph

### Function definitions

In [92]:
from myfunctions import *

### Compute weights *w*

In [93]:
center = ( center_x, center_y, center_z)
max_radii = ( max_r_x, max_r_y, max_r_z )
min_radii = ( min_r_x, min_r_y, min_r_z )
w, w_tilde = compute_weights( image, center, K, max_radii, col_vectors, min_radii=min_radii )

### Build and solve flow network

In [94]:
g,nodes = build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde )
maxval = g.maxflow()

### Visualize results

In [95]:
size_s_comp = 0
size_t_comp = 0
for n in nodes:
    seg = g.get_segment(n)
    if seg == 0:
        size_s_comp += 1
    else:
        size_t_comp += 1
    #print "Segment of the node ",n,":", seg
print "Size of s and t component: ", size_s_comp, size_t_comp

Size of s and t component:  4921 10439


In [105]:
def create_dot_annotatation_image():
    # imgout = np.zeros(image.shape)
    # valtoplace = 1
    imgout = np.copy(image) #np.zeros_like(image)
    valtoplace = 1.0*np.max(image)
    for i in range(num_columns):
        for k in range(K):
            x = int(center_x + col_vectors[i,0] * min_r_x + col_vectors[i,0] * k/float(K) * (max_r_x-min_r_x))
            y = int(center_y + col_vectors[i,1] * min_r_y + col_vectors[i,1] * k/float(K) * (max_r_y-min_r_y))
            z = int(center_z + col_vectors[i,2] * min_r_z + col_vectors[i,2] * k/float(K) * (max_r_z-min_r_z))
            seg = g.get_segment(i*K+k)
            if seg == 0:
                if x>=0 and y>=0 and z>=0 and \
                   x<image.shape[2] and y<image.shape[1] and z<image.shape[0]:
                    imgout[z,y,x] = valtoplace
    return imgout

# IDEE: draw line to all neighbors (at height k) along 3d bresenham

In [106]:
volwin = volshow(create_dot_annotatation_image(), stackUnits = [1.,1.,4.], raise_window=False)

# Create video frames 

In [103]:
frames = []
f = range(100,431,10)
for i, center_x in enumerate(f):
    
    center_y = 280
    center_z = 26
    center = ( center_x, center_y, center_z)

    w, w_tilde = compute_weights( image, center, K, max_radii, col_vectors, min_radii=min_radii )
    g,nodes = build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde )
    maxval = g.maxflow()
    
    frames.append( create_dot_annotatation_image() )
    print '.',
    
f = range(280,200,-10)
for i, center_y in enumerate(f):
    
    center_x = 430
    center_z = 26
    center = ( center_x, center_y, center_z)

    w, w_tilde = compute_weights( image, center, K, max_radii, col_vectors, min_radii=min_radii )
    g,nodes = build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde )
    maxval = g.maxflow()
    
    frames.append( create_dot_annotatation_image() )
    print '.',

print 'done'

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done


In [83]:
volwin = volshow(np.array(frames), stackUnits = [1.,1.,4.], raise_window=True)

/Users/jug/Repositories/GIT/BobSeg/video


In [99]:
imsave('video/center_sampling_maddox_spim_dots.tif', np.array(frames))